In [ ]:

import pandas as pd0
from scipy.stats import ttest_ind0

In [6]:
# load the dataset
df_merge = pd.read_csv('./data/annotation/rave_disagreement.tsv',sep='\t'
                       )

In [7]:
##  preprocess proxy variables

#JudgeSplit:
''' (If the case decision is not a unanimous vote > It is a split vote)'''
df_merge['JudgeSplit'] = (df_merge['unanimous'] != 1).astype(int)

#Keycase:
''' whether the case is listed as keycase by ECtHR'''
df_merge['Keycase'] = (df_merge['key_case'] == 1).astype(int)

#OmitAlleg:
'''Subsidiary allegations absent in the original HUDOC metadata'''
df_merge['OmitAlleg'] = (df_merge['meta'] == 1).astype(int)

#DiffPred
'''Difference in Outcome Prediction'''
df_merge['DiffPred'] = (df_merge['pred_i'] != df_merge['pred_o']).astype(int)

#DiffFam
'''Difference in Case Familiarity'''
df_merge['DiffFam'] = (df_merge['known_i'] != df_merge['known_o']).astype(int)

#NumPar
'''utilize number of paragraphs'''
# df_merge['numPar']







'utilize number of paragraphs'

In [8]:
## t-test


def significatn_test(df, feature):
  # feature: binary proxy variables of the possible sources of disagreeement


# group the DataFrame by the values in column feature
  grouped = df.groupby(feature)

  # # get the groups for 0 and 1
  group0 = grouped.get_group(0)["interAnno_k_1"]
  group1 = grouped.get_group(1)["interAnno_k_1"]

  # get the mean of each group
  mean_0 = round(group0.mean(),3)
  mean_1 = round(group1.mean(),3)

  # # perform t-test on the two groups
  t_statistic, p_value = ttest_ind(group0, group1, equal_var=True)
  print(f'p_value: {p_value}')
  print(f't_statistic: {t_statistic}')

  print(f'mean_0: {mean_0}')
  print(f'mean_1: {mean_1}')

  # print results
  if p_value < 0.05:
    if mean_0 > mean_1:
      print(f"The mean of {feature} 0 is significantly bigger than that of group 1.")
    else:
      print(f"The mean of {feature} 1 is significantly bigger than that of group 0.")
  else:
      print("There is no significant difference between the means of group 0 and group 1.")


In [9]:
# Main

'''statistical association between proxy variables and expert’s IAA score.'''
proxy_variables = ['JudgeSplit','Keycase','OmitAlleg','DiffPred','DiffFam']

for proxy in proxy_variables:
  significatn_test(df_merge, proxy)

p_value: 0.0028095172062429113
t_statistic: -3.095729823931687
mean_0: 0.218
mean_1: 0.344
The mean of JudgeSplit 1 is significantly bigger than that of group 0.
p_value: 0.8555247997504345
t_statistic: -0.18273748189384495
mean_0: 0.242
mean_1: 0.249
There is no significant difference between the means of group 0 and group 1.
p_value: 8.500157514472284e-06
t_statistic: -4.800667416613973
mean_0: 0.086
mean_1: 0.278
The mean of OmitAlleg 1 is significantly bigger than that of group 0.
p_value: 0.03883616037592681
t_statistic: -2.104942958915564
mean_0: 0.225
mean_1: 0.315
The mean of DiffPred 1 is significantly bigger than that of group 0.
p_value: 0.6508758702443435
t_statistic: -0.4544715907860136
mean_0: 0.24
mean_1: 0.26
There is no significant difference between the means of group 0 and group 1.
